In [705]:
import numpy as np
import math
from IPython.core.display import display, HTML
from random import randint

In [701]:
class Grid:
    def __init__(self, rows, columns, labelVec = None):
        self.rows = rows
        self.columns = columns
        
        if(labelVec == None):
            self.labelVec = list(range(1,rows*columns+1))
        else:
            self.labelVec = labelVec.copy()
        
    def display(self):
        table = "<table>{0}</table>"
        row = "<tr>{0}</tr>"
        cell = "<td>{0}</td>"

        cells = ""
        rows = "" 
        for i in range(1,self.rows*self.columns+1):
            cells += cell.format(self.labelVec[i-1])
            if((i%self.rows) == 0):
                rows += row.format(cells)
                cells = ""

        display(HTML(table.format(rows)))
        
    def __repr__(self):
        self.display()
        return "{0}x{1} grid".format(self.rows,self.columns)
    
    def D(self):
        # (1+2,2+4,5+2)->(3,6,7) (3x3)
        # (1+3,2+6,3+6,6+6,7+6,11+3)->(4,8,9,12,13,14) (4x4)
        nTerms = math.floor((self.rows*self.columns - self.rows)/2)
        extremeAddened = self.rows-1
        midAddened = extremeAddened*2
        
        self.labelVec[1],self.labelVec[3] = self.labelVec[3],self.labelVec[1]
        self.labelVec[2],self.labelVec[6] = self.labelVec[6],self.labelVec[2]
        self.labelVec[5],self.labelVec[7] = self.labelVec[7],self.labelVec[5]
             
        return self
    
    def C(self):
        # (0,1,3)->(5,7,8)
        self.labelVec[0],self.labelVec[8] = self.labelVec[8],self.labelVec[0]
        self.labelVec[1],self.labelVec[5] = self.labelVec[5],self.labelVec[1]
        self.labelVec[3],self.labelVec[7] = self.labelVec[7],self.labelVec[3]
             
        return self
    
    # Obviously could be simplified if we just used nested loops, not sure why
    # I felt the need to do it in this way. Maybe the math is getting to me. 
    def V(self):
        if(self.rows % 2 != 0 or self.rows % 2 != 0):
            itter = self.rows*math.floor( math.sqrt(self.rows*self.columns) / 2 )
        else:
            itter = self.rows*math.floor(self.columns/2)
        for i in range(itter):
            overLoc = (self.rows-1+self.rows*(i%self.rows))-math.floor(i/self.rows)
            leftLoc = math.floor(i/self.rows)+i%self.rows+((self.rows-1)*(i%self.rows))
            tmp = self.labelVec[leftLoc]
            
            self.labelVec[leftLoc] = self.labelVec[overLoc]
            self.labelVec[overLoc] = tmp
        
        return self
                

    
    # Horizontal Symmetry 
    def H(self):
        if(self.rows % 2 != 0 or self.rows % 2 != 0):
            itter = self.rows+self.rows*math.floor( math.sqrt(self.rows*self.columns) / 2 )
        else:
            itter = self.rows*math.floor(self.columns/2)
            
        for i in range(itter):
            overLoc = (self.rows*self.columns-self.rows*(math.floor(i/self.rows)+1)+(i%(self.rows)))
            tmp = self.labelVec[i]
            self.labelVec[i] = self.labelVec[overLoc]
            self.labelVec[overLoc] = tmp
            
        return self

In [702]:
g = Grid(3,3)
g

1,2,3
4,5,6
7,8,9


3x3 grid

In [828]:
z = Grid(3,3)
currentPath = []
longestPath = []
currentPathOps = []
longestPathOps = []

filterNum = 7
filterLs = []

for i in range(1000000):
    roll = randint(1,4)
    if roll == 1:
        z.H()
        currentPathOps.append("H")
    elif roll == 2:
        z.V()
        currentPathOps.append("V")
    elif roll == 3:
        z.D()
        currentPathOps.append("D")
    else:
        z.C()
        currentPathOps.append("C")
    
    addToCurrent = True
    
    for i in currentPath:
        if i == z.labelVec:
            # We have reached a redundancy.
            if(len(currentPath)>len(longestPath)):
                # We have found the longest path so far.
                # Pop the redundant path and add to longest path. 
                currentPath.pop()
                del currentPathOps[-2:]
                
                longestPath = currentPath
                longestPathOps = currentPathOps
                
            
                if(len(currentPath) == filterNum):
                    filterLs.append(currentPathOps)
                
            z = Grid(3,3)
            currentPath = []
            currentPathOps = []
            addToCurrent = False
    
    if(addToCurrent):
        currentPath.append(z.labelVec.copy())

In [829]:
len(longestPath)

7

In [830]:
longestPath

[[7, 8, 9, 4, 5, 6, 1, 2, 3],
 [1, 2, 3, 4, 5, 6, 7, 8, 9],
 [9, 6, 3, 8, 5, 2, 7, 4, 1],
 [3, 6, 9, 2, 5, 8, 1, 4, 7],
 [1, 4, 7, 2, 5, 8, 3, 6, 9],
 [9, 8, 7, 6, 5, 4, 3, 2, 1],
 [3, 2, 1, 6, 5, 4, 9, 8, 7]]

In [831]:
longestPathOps

['H', 'H', 'C', 'V', 'H', 'C', 'H']

In [832]:
filterLs

[['D', 'V', 'C', 'D', 'C', 'H', 'C'],
 ['H', 'D', 'V', 'H', 'D', 'V', 'C'],
 ['D', 'V', 'C', 'H', 'V', 'C', 'V'],
 ['H', 'V', 'H', 'C', 'V', 'H', 'V'],
 ['V', 'C', 'V', 'C', 'D', 'V', 'C'],
 ['V', 'D', 'V', 'D', 'V', 'D', 'V'],
 ['D', 'D', 'V', 'H', 'V', 'D', 'H'],
 ['C', 'H', 'D', 'C', 'H', 'C', 'H'],
 ['H', 'H', 'C', 'H', 'C', 'H', 'C'],
 ['C', 'C', 'D', 'V', 'D', 'C', 'D'],
 ['H', 'D', 'V', 'D', 'V', 'D', 'V'],
 ['D', 'D', 'C', 'D', 'V', 'C', 'D'],
 ['H', 'C', 'V', 'D', 'C', 'D', 'V'],
 ['D', 'H', 'V', 'H', 'D', 'H', 'V'],
 ['H', 'C', 'H', 'C', 'H', 'V', 'C'],
 ['C', 'D', 'V', 'C', 'D', 'C', 'V'],
 ['D', 'V', 'H', 'D', 'V', 'C', 'D'],
 ['D', 'V', 'C', 'D', 'V', 'C', 'H'],
 ['D', 'C', 'V', 'H', 'C', 'V', 'D'],
 ['H', 'C', 'D', 'V', 'D', 'V', 'H'],
 ['V', 'V', 'H', 'C', 'V', 'D', 'C'],
 ['H', 'H', 'D', 'H', 'D', 'H', 'D'],
 ['D', 'H', 'D', 'V', 'C', 'D', 'V'],
 ['D', 'H', 'D', 'H', 'D', 'C', 'H'],
 ['V', 'H', 'V', 'D', 'H', 'C', 'D'],
 ['C', 'C', 'D', 'H', 'D', 'H', 'V'],
 ['D', 'H', 

In [834]:
g = Grid(3,3)
g.V().D().V().D().V().D().V()

1,4,7
2,5,8
3,6,9


3x3 grid

In [454]:
5/2

2.5

In [79]:
del g 